# 打開or關閉瀏覽器，完成時間一樣，150頁花費約50分鐘
# 記得改成你想要的關鍵字

In [1]:
# 搜尋關鍵字 - python / 數據分析 / 商業分析 / tableau / power bi
keyword = 'keras'
url = 'https://www.104.com.tw/jobs/search/?keyword={}'.format(keyword)

In [2]:
# 使用webdriver-manager，自動下載chromedriver，避免每次版本控制的問題
# pip install webdriver-manager

In [3]:
import re
import requests
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor
import winsound

In [4]:
start = time.time()

In [5]:
chrome_options = Options()
chrome_options.add_argument('--headless')           # 指定headless，并且禁用gpu
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('window-size=1920,1080') # 設定瀏覽器尺寸，假如無法滾動的話

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(url)
driver.maximize_window()

# 抓取總共有幾頁
text = driver.find_element(By.CSS_SELECTOR, '#js-job-header > div.b-float-right > label:nth-child(1) > select').text
total_page = int(re.findall(r'\d+', text)[-1])

In [6]:
# 自動翻頁
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        # Scroll down to bottom
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [7]:
# 遇到手動翻頁時，自動判斷button的css selector數字，方便後續點擊正確的按鈕
# 如果無法翻頁，需要調整數字300-4000，要涵蓋更大的範圍
# 最多150頁*每頁22筆 = 3300，再加上第16頁的326，所以設定4000，如果未來頁數增加，數字要再調大
def button():
    for i in range(300, 4000):
        try:
            if '手動載入' in driver.find_element(by=By.CSS_SELECTOR, value='#js-job-content > div:nth-child({}) > button'.format(i)).text:
                return int(i)
        except:
            pass
    
    # 如果沒有出現手動載入按鈕，就回傳0
    return int(0)

In [8]:
# 自動翻頁只能到15頁，超過15頁要手動載入
# 0.5秒翻頁太快，會爬不到所有資料
scroll(driver, 1)

In [9]:
# 手動載入，自動翻頁
for i in range(button(), total_page*22+1, 22):
    # 如果不需要翻頁，就跳出迴圈
    if button() == 0:
        break
    # 如果需要翻頁，就繼續點擊手動載入按鈕
    try:
        driver.find_element(by=By.CSS_SELECTOR, value='div:nth-child({}) > button'.format(i)).click()
    except:
        pass

    scroll(driver, 1)

In [10]:
company = []
industry = []
job_name = []
location = []
salary = []
requirement = []
education = []
descirption = []
link = []

In [11]:
# 同步處理
# 每一頁的職缺有22個，最多爬取150頁，3300筆資料，再扣掉廣告
for i in range(1, total_page*22+1):
    try:
        job_name_tag = driver.find_element(By.CSS_SELECTOR, "article:nth-child({}) > div.b-block__left > h2 > a".format(i)).text
        job_name.append(job_name_tag)

        link_tag = driver.find_element(By.CSS_SELECTOR, "article:nth-child({}) > div.b-block__left > h2 > a".format(i)).get_attribute("href")
        link.append(link_tag)

        location_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(1)'.format(i)).text
        location.append(location_tag)

        requirement_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(3)'.format(i)).text
        requirement.append(requirement_tag)

        try:
            company_name_tag = driver.find_element(By.CSS_SELECTOR, '#js-job-content > article:nth-child({}) > div.b-block__left > ul:nth-child(2) > li:nth-child(2) > a'.format(i)).text
            company.append(company_name_tag)
        except:
            company.append("Null")

        try:
            industry_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul:nth-child(2) > li:nth-child(4)'.format(i)).text
            industry.append(industry_tag)
        except:
            industry.append("Null")

        try:
            education_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(5)'.format(i)).text
            education.append(education_tag)
        except:
            education.append("Null")

        try:
            descirption_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > p'.format(i)).text
            descirption.append(descirption_tag)
        except:
            descirption.append("Null")

        try:
            salary_tag = driver.find_element(By.CSS_SELECTOR, '#js-job-content > article:nth-child({}) > div.b-block__left > div > a:nth-child(1)'.format(i)).text
            salary.append(salary_tag)
        except:
            salary_tag = driver.find_element(By.CSS_SELECTOR, "#js-job-content > article:nth-child({}) > div.b-block__left > div > span".format(i)).text
            salary.append(salary_tag)
            
        print("第" + str(i) + '職缺爬完')

    except:
        pass
        print("第" + str(i) + '職缺不存在 or 廣告')

第1職缺爬完
第2職缺爬完
第3職缺爬完
第4職缺爬完
第5職缺爬完
第6職缺爬完
第7職缺爬完
第8職缺爬完
第9職缺爬完
第10職缺爬完
第11職缺不存在 or 廣告
第12職缺爬完
第13職缺爬完
第14職缺爬完
第15職缺爬完
第16職缺爬完
第17職缺爬完
第18職缺爬完
第19職缺爬完
第20職缺爬完
第21職缺不存在 or 廣告
第22職缺爬完
第23職缺不存在 or 廣告
第24職缺爬完
第25職缺爬完
第26職缺爬完
第27職缺爬完
第28職缺爬完
第29職缺爬完
第30職缺爬完
第31職缺爬完
第32職缺不存在 or 廣告
第33職缺爬完
第34職缺爬完
第35職缺爬完
第36職缺爬完
第37職缺爬完
第38職缺爬完
第39職缺爬完
第40職缺爬完
第41職缺爬完
第42職缺不存在 or 廣告
第43職缺爬完
第44職缺爬完
第45職缺爬完
第46職缺不存在 or 廣告
第47職缺爬完
第48職缺爬完
第49職缺爬完
第50職缺爬完
第51職缺爬完
第52職缺爬完
第53職缺不存在 or 廣告
第54職缺爬完
第55職缺爬完
第56職缺爬完
第57職缺爬完
第58職缺爬完
第59職缺爬完
第60職缺爬完
第61職缺爬完
第62職缺爬完
第63職缺不存在 or 廣告
第64職缺爬完
第65職缺爬完
第66職缺爬完
第67職缺爬完
第68職缺爬完
第69職缺不存在 or 廣告
第70職缺爬完
第71職缺爬完
第72職缺爬完
第73職缺爬完
第74職缺不存在 or 廣告
第75職缺爬完
第76職缺爬完
第77職缺爬完
第78職缺爬完
第79職缺爬完
第80職缺爬完
第81職缺爬完
第82職缺爬完
第83職缺爬完
第84職缺不存在 or 廣告
第85職缺爬完
第86職缺爬完
第87職缺爬完
第88職缺爬完
第89職缺爬完
第90職缺爬完
第91職缺爬完
第92職缺不存在 or 廣告
第93職缺爬完
第94職缺爬完
第95職缺不存在 or 廣告
第96職缺爬完
第97職缺爬完
第98職缺爬完
第99職缺爬完
第100職缺爬完
第101職缺爬完
第102職缺爬完
第103職缺爬完
第104職缺爬完
第105職缺不存在 or 廣告
第106職缺爬完
第107職缺爬完
第108職缺爬完
第109職缺爬完
第110職缺爬完
第111職缺爬完
第112職缺爬完
第1

In [12]:
print(len(company))
print(len(industry))
print(len(job_name))
print(len(location))
print(len(salary))
print(len(requirement))
print(len(education))   
print(len(descirption)) 
print(len(link))

157
157
157
157
157
157
157
157
157


In [25]:
data = {
    'company':      company,
    'industry':     industry,
    'job_name':     job_name,
    'location':     location,
    'salary':       salary,
    'requirement':  requirement,
    'education':    education,
    'descirption':  descirption,
    'link':         link,
}

df = pd.DataFrame(data)
# link一樣工作內容就會一樣，刪除重複的工作
df.drop_duplicates(subset='link',inplace=True)   
df.to_excel('./result/104_{}.xlsx'.format(keyword), sheet_name= keyword)

In [ ]:
end = time.time()
print('完成共花費',round((end - start)/60,2),'分')

完成共花費 46.2 分


In [ ]:
winsound.Beep(300,500)
winsound.Beep(360,500)
winsound.Beep(500,500)
winsound.PlaySound('alert', winsound.SND_ASYNC)

In [ ]:
# # 非同步處理，薪資順序會亂掉，不建議使用，時間並沒有快多少
# def scrape_job_details(i):
#     try:
#         job_name_tag = driver.find_element(By.CSS_SELECTOR, "article:nth-child({}) > div.b-block__left > h2 > a".format(i)).text
#         job_name.append(job_name_tag)

#         link_tag = driver.find_element(By.CSS_SELECTOR, "article:nth-child({}) > div.b-block__left > h2 > a".format(i)).get_attribute("href")
#         link.append(link_tag)

#         location_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(1)'.format(i)).text
#         location.append(location_tag)

#         requirement_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(3)'.format(i)).text
#         requirement.append(requirement_tag)

#         try:
#             company_name_tag = driver.find_element(By.CSS_SELECTOR, '#js-job-content > article:nth-child({}) > div.b-block__left > ul:nth-child(2) > li:nth-child(2) > a'.format(i)).text
#             company.append(company_name_tag)
#         except:
#             company.append("Null")

#         try:
#             industry_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul:nth-child(2) > li:nth-child(4)'.format(i)).text
#             industry.append(industry_tag)
#         except:
#             industry.append("Null")

#         try:
#             education_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > ul.b-list-inline.b-clearfix.job-list-intro.b-content > li:nth-child(5)'.format(i)).text
#             education.append(education_tag)
#         except:
#             education.append("Null")

#         try:
#             descirption_tag = driver.find_element(By.CSS_SELECTOR, 'article:nth-child({}) > div.b-block__left > p'.format(i)).text
#             descirption.append(descirption_tag)
#         except:
#             descirption.append("Null")

#         try:
#             salary_tag = driver.find_element(By.CSS_SELECTOR, '#js-job-content > article:nth-child({}) > div.b-block__left > div > a:nth-child(1)'.format(i)).text
#             salary.append(salary_tag)
#         except:
#             salary_tag = driver.find_element(By.CSS_SELECTOR, "#js-job-content > article:nth-child({}) > div.b-block__left > div > span".format(i)).text
#             salary.append(salary_tag)
#         print("第" + str(i) + '職缺爬完')

#     except:
#         pass
#         print("第" + str(i) + '職缺不存在 or 廣告')

# # Create a ThreadPoolExecutor with max_workers set to the number of threads you want to use
# with ThreadPoolExecutor(max_workers=8) as executor:
#     # Submit the scraping tasks to the executor
#     futures = [executor.submit(scrape_job_details, i) for i in range(1, total_page*22+1)]

#     # Wait for all tasks to complete
#     for future in futures:
#         future.result()
